<a href="https://colab.research.google.com/github/mansi-98/Real_time_emotion_detection/blob/master/Emotion_Anaysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [103]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## **Importing** **Libraries** 

In [0]:
import sys, os
import pandas as pd
import numpy as np

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization,AveragePooling2D
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.utils import np_utils

In [0]:
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import RandomizedSearchCV, KFold, cross_val_score
from keras.optimizers import Adam
from keras import regularizers
from sklearn.model_selection import GridSearchCV

#Loading the FER2013 dataset

In [106]:

df=pd.read_csv('/content/gdrive/My Drive/Realtime-Emotion-Detection/fer2013.csv')
# print(df.info())
# print(df["Usage"].value_counts())

print(df.head())

   emotion                                             pixels     Usage
0        0  70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...  Training
1        0  151 150 147 155 148 133 111 140 170 174 182 15...  Training
2        2  231 212 156 164 174 138 161 173 182 200 106 38...  Training
3        4  24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...  Training
4        6  4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...  Training


# Data preparation


In [0]:
X_train,train_y,X_test,test_y=[],[],[],[]

for index, row in df.iterrows():
    val=row['pixels'].split(" ")
    try:
        if 'Training' in row['Usage']:
           X_train.append(np.array(val,'float32'))
           train_y.append(row['emotion'])
        elif 'PublicTest' in row['Usage']:
           X_test.append(np.array(val,'float32'))
           test_y.append(row['emotion'])
    except:
        print(f"error occured at index :{index} and row:{row}")



In [0]:
num_features = 64
num_labels = 7
batch_size = 64
epochs = 50
width, height = 48, 48


X_train = np.array(X_train,'float32')
train_y = np.array(train_y,'float32')
X_test = np.array(X_test,'float32')
test_y = np.array(test_y,'float32')

train_y=np_utils.to_categorical(train_y, num_classes=num_labels)
test_y=np_utils.to_categorical(test_y, num_classes=num_labels)


In [0]:
#cannot produce
#normalizing data between oand 1
X_train -= np.mean(X_train, axis=0)
X_train /= np.std(X_train, axis=0)

X_test -= np.mean(X_test, axis=0)
X_test /= np.std(X_test, axis=0)

X_train = X_train.reshape(X_train.shape[0], 48, 48, 1)

X_test = X_test.reshape(X_test.shape[0], 48, 48, 1)


# Cnn Model

In [0]:
##designing the cnn
#1st convolution layer
model = Sequential()

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(X_train.shape[1:])))
model.add(Conv2D(64,kernel_size= (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
model.add(Dropout(0.5))

#2nd convolution layer
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
model.add(Dropout(0.5))

#3rd convolution layer
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))

model.add(Flatten())

#fully connected neural networks
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(num_labels, activation='softmax'))

In [111]:
model.summary()

Model: "sequential_28"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_163 (Conv2D)          (None, 46, 46, 64)        640       
_________________________________________________________________
conv2d_164 (Conv2D)          (None, 44, 44, 64)        36928     
_________________________________________________________________
batch_normalization_1 (Batch (None, 44, 44, 64)        256       
_________________________________________________________________
max_pooling2d_82 (MaxPooling (None, 22, 22, 64)        0         
_________________________________________________________________
dropout_109 (Dropout)        (None, 22, 22, 64)        0         
_________________________________________________________________
conv2d_165 (Conv2D)          (None, 20, 20, 64)        36928     
_________________________________________________________________
conv2d_166 (Conv2D)          (None, 18, 18, 64)      

In [112]:
#Compliling the model
model.compile(loss=categorical_crossentropy,
              optimizer=Adam(),
              metrics=['accuracy'])

#Training the model
model.fit(X_train, train_y,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, test_y),
          shuffle=True)

Train on 28709 samples, validate on 3589 samples
Epoch 1/50
28709/28709 [==============================] - 16s 552us/step - loss: 1.7096 - accuracy: 0.3230 - val_loss: 1.5478 - val_accuracy: 0.3876
Epoch 2/50
28709/28709 [==============================] - 15s 521us/step - loss: 1.4709 - accuracy: 0.4285 - val_loss: 1.4372 - val_accuracy: 0.4478
Epoch 3/50
28709/28709 [==============================] - 15s 515us/step - loss: 1.3558 - accuracy: 0.4793 - val_loss: 1.2813 - val_accuracy: 0.5130
Epoch 4/50
28709/28709 [==============================] - 15s 517us/step - loss: 1.2883 - accuracy: 0.5032 - val_loss: 1.2868 - val_accuracy: 0.5166
Epoch 5/50
28709/28709 [==============================] - 15s 511us/step - loss: 1.2257 - accuracy: 0.5321 - val_loss: 1.2191 - val_accuracy: 0.5350
Epoch 6/50
28709/28709 [==============================] - 15s 513us/step - loss: 1.1753 - accuracy: 0.5514 - val_loss: 1.2152 - val_accuracy: 0.5444
Epoch 7/50
28709/28709 [==============================] -

In [113]:
loss, accuracy = model.evaluate(X_test, test_y)
print(accuracy)#accuarcy with relu function=0.600724458694458 in 50 epoch
#accuracy with elu function = 0.569796621799469 in 50 epoch

3589/3589 [==============================] - 1s 192us/step
0.600724458694458


In [35]:
print(np.shape(X_train))
print(np.shape(train_y))
print(np.shape(X_test))
print(np.shape(test_y))

(28709, 48, 48, 1)
(28709, 7)
(3589, 48, 48, 1)
(3589, 7)


# Saving The Model

In [0]:
#Saving the  model to  use it later on
fer_json = model.to_json()
with open("fer.json", "w") as json_file:
    json_file.write(fer_json)
model.save_weights("fer.h5")

In [0]:
def create_model(learning_rate=0.01, activation='relu'):

  model = Sequential()

  model.add(Conv2D(64, kernel_size=(3, 3), activation = activation, input_shape=(X_train.shape[1:])))
  model.add(Conv2D(64,kernel_size= (3, 3),activation = activation))
# model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
  model.add(Dropout(0.5))

#2nd convolution layer
  model.add(Conv2D(64, (3, 3), activation = activation))
  model.add(Conv2D(64, (3, 3), activation = activation))
# model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
  model.add(Dropout(0.5))

#3rd convolution layer
  model.add(Conv2D(128, (3, 3), activation = activation))
  model.add(Conv2D(128, (3, 3), activation = activation))
# model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))

  model.add(Flatten())

#fully connected neural networks
  model.add(Dense(1024, activation = activation))
  model.add(Dropout(0.2))
  model.add(Dense(1024,activation = activation))
  model.add(Dropout(0.2))

  model.add(Dense(num_labels, activation='softmax'))

# model.summary()

#Compliling the model
  model.compile(loss=categorical_crossentropy,optimizer=Adam(),metrics=['accuracy'])
  return model

# Hyperparameter tuning


In [0]:

# Create a KerasClassifier
model = KerasRegressor(build_fn = create_model,
                       verbose = 0)

# Define the parameters to try out
params = {'activation': ["relu","elu"],
          'batch_size': [32,64], 
          'epochs': [10,50]}
          # 'learning_rate': [0.01, 0.001, 0.0001,0.00001]}

# Create a randomize search cv object passing in the parameters to try
random_search = RandomizedSearchCV(model,
                                   param_distributions = params,
                                   cv = KFold(10))


In [0]:
random_search_results = random_search.fit(X_train, train_y)

In [0]:
print("Best Score: ", random_search_results.best_score_, "and Best Params: ", random_search_results.best_params_)